In [ ]:
import os
import torch
import numpy as np
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import rootutils
import torchvision.transforms as T
from sklearn.metrics import accuracy_score, classification_report

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from helpers.inference.inference_onnx import InferenceOnnx
from helpers.inference.inference_trt import InferenceTensorRT

from src.data.components.preprocessing.preproc_pipeline_manager import (
    PreprocessingPipeline,
)
from src.data.components.preprocessing.preproc_strategy_split import SplitStep
from src.data.components.preprocessing.preproc_strategy_tile import TilingStep
from src.data.classification_datamodule import ClassificationDataModule

# loading .env variables
load_dotenv()

#### Initialize dataloader

In [2]:
val_test_transforms = T.Compose(
    [
        T.ToTensor(),
    ]
)

split_step = SplitStep(
    split_ratio=[0.7, 0.2, 0.1],
    seed=42,
    merge_classes=True,
)

tiling_step = TilingStep(
    tile_size=[224, 224],
    min_defective_area_th=0.1,
    discard_background_th=0.0,
    overlap=10,
    contour_iter_step_size=10,
    iterate_over_defective_areas=True,
)

preprocessing_pipeline = PreprocessingPipeline([split_step, tiling_step])

# Initialize the DataModule
data_dir = os.environ.get('seats_data_path')
data_module = ClassificationDataModule(
    data_dir=data_dir,
    preprocessing_pipeline=preprocessing_pipeline,
    batch_size=1,
    num_workers=3,
    pin_memory=False,
    val_test_transforms=val_test_transforms,
)

# Set up the test dataset
data_module.prepare_data()
data_module.setup(stage='test')
test_loader = data_module.test_dataloader()

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def evaluate(model):
    results = []
    for batch in tqdm(test_loader, desc='Evaluating'):
        inputs, labels = batch

        if isinstance(model, torch.nn.Module):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predictions = (outputs[0] > 0.5).int().cpu().numpy()
            labels = labels.cpu().numpy()

        elif isinstance(model, InferenceOnnx) or isinstance(model, InferenceTensorRT):
            inputs = inputs.numpy()
            labels = labels.numpy()
            outputs = model(inputs)
            predictions = (outputs[0] > 0.5).astype(int)

        results.append((predictions, labels))

    # Metrics calculation
    all_preds, all_labels = zip(*results)
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    print(classification_report(all_labels, all_preds))

In [ ]:
inference_onnx = InferenceOnnx('efficientnet_v2_s_downscaled_pcb.onnx')
evaluate(inference_onnx)

In [ ]:
inference_trt = InferenceTensorRT('efficientnet_v2_s_downscaled_pcb_fp32.trt')
evaluate(inference_trt)

In [ ]:
from src.models.components.cnn_cam_multihead import CNNCAMMultihead
from src.models.components.nn_utils import weight_load

model = CNNCAMMultihead(
    backbone='torchvision.models/efficientnet_v2_s',
    return_node='features.6.0.block.0',
    multi_head=True,
).to(device)
weights = weight_load(
    ckpt_path='../trained_models/models--DeepVisionXplain--efficientnet_v2_s_downscaled_pcb/',
    weights_only=True,
)
model.load_state_dict(weights)
model.eval()

with torch.no_grad():
    evaluate(model)